In [ ]:
!pip install -q gradio

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googletrans 4.0.0rc1 requires httpx==0.13.3, but you have httpx 0.27.0 which is incompatible.


In [ ]:
!pip install soundfile

In [ ]:
!pip install googletrans==4.0.0-rc1

  Using cached httpx-0.13.3-py3-none-any.whl (55 kB)
  Using cached httpcore-0.9.1-py3-none-any.whl (42 kB)
  Using cached h11-0.9.0-py2.py3-none-any.whl (53 kB)
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Successfully uninstalled h11-0.14.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 1.0.5
    Uninstalling httpcore-1.0.5:
      Successfully uninstalled httpcore-1.0.5
  Attempting uninstall: httpx
    Found existing installation: httpx 0.27.0
    Uninstalling httpx-0.27.0:
      Successfully uninstalled httpx-0.27.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 4.25.0 requires httpx>=0.24.1, but you have httpx 0.13.3 which is incompatible.
gradio-client 0.15.0 requires httpx>=0.24.1, but you have httpx 0.13.3 which is incompatible.


In [ ]:
!pip install TTS API

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 938.0/938.0 kB 18.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 84.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install transformers
!pip install --upgrade numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 46.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.0
    Uninstalling numpy-1.22.0:
      Successfully uninstalled numpy-1.22.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tts 0.22.0 requires numpy==1.22.0; python_version <= "3.10", but you have numpy 1.26.4 which is incompatible.
gradio 4.25.0 requires httpx>=0.24.1, but you have httpx 0.13.3 which is incompatible.


In [ ]:
import os
# Restart the runtime
os._exit(0)

In [ ]:

import csv
import librosa
import torch
from transformers import pipeline
from googletrans import Translator

# Initialize the transcription pipeline
model_name = "openai/whisper-large-v3"
device = 0 if torch.cuda.is_available() else -1
pipe = pipeline("automatic-speech-recognition", model=model_name, device=device)

def translate_audio(audio_path, target_sr=16000, max_duration=None, task='translate',source_language="ar",target_language="en"):
    # Load the audio file
    audio, sr = librosa.load(audio_path, sr=target_sr)
    # Trim the audio to max_duration if specified
    if max_duration:
        max_samples = max_duration * target_sr
        if len(audio) > max_samples:
            audio = audio[:max_samples]
    if source_language == "ar":
        translation_result = pipe(audio, generate_kwargs={"task": task, "language":target_language})
        translation_text = translation_result["text"]
    elif source_language == "en":
      transcribe_result = pipe(audio, generate_kwargs={"task": "transcribe", "language":source_language})
      transcribe_result = transcribe_result["text"]
      # Translator object
      translator = Translator()
      translated_text = translator.translate(transcribe_result, dest='ar')
      translation_text = translated_text.text


    return translation_text


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import gradio as gr
from TTS.api import TTS
# Initialize your TTS system
# getting the latest XTTS_v2
tts = TTS("xtts", gpu=True)
def gradio_process(uploaded_audio_path, source_language_, target_language_):
    global latest_translation
    # Audio transcription and translation.
    translation = translate_audio(audio_path=uploaded_audio_path, source_language=source_language_, target_language=target_language_)
    latest_translation = translation

    if not latest_translation:
        return "No text to clone. Please translate first.", ("", "audio/wav")

    # Perform voice cloning
    output_audio_path = "/tmp/cloned_voice_output.wav"
    tts.tts_to_file(text=latest_translation,
                    file_path=output_audio_path,
                    speaker_wav=uploaded_audio_path,
                    language=target_language_)

    # Return the translation and the path to the cloned audio file
    return latest_translation, output_audio_path

# Gradio interface
iface = gr.Interface(
    fn=gradio_process,
    inputs=[
        gr.Audio(label="Upload your audio file (WAV format)", type="filepath"),
        gr.Dropdown(choices=["en", "ar"], label="Source Language"),
        gr.Dropdown(choices=["en", "ar"], label="Target Language"),
        #gr.Radio(choices=["Translate", "Clone"], label="Action"),
        #gr.Textbox(label="Enter translation here if you wish to modify it before cloning"),
    ],
    outputs=[
        gr.Textbox(label="Translated Text"),
        gr.Audio(label="Cloned Voice Audio"),
    ],
    title="DupAI: Audio Translation and Voice Cloning",
    description="Upload an audio file and select the source and target language to translate. You can then clone the voice in the translated language."
)

iface.launch(debug=True)

/usr/local/lib/python3.10/dist-packages/TTS/api.py:70: UserWarning: `gpu` will be deprecated. Please use `tts.to(device)` instead.
  warnings.warn("`gpu` will be deprecated. Please use `tts.to(device)` instead.")


 > Using model: xtts
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://ef5acfd7ba3b45e814.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


 > Text splitted to sentences.
["The way of Ibn Ziyad to the world, Julian, the ruler of Sabta I greet you with the greeting of Islam We have received what you have offered us We are not students of a land or a kingdom On these conditions The merchants of Sabta should enter the Tanjah with their trade safe They have the covenant of God and the authority of their leader People believe in their religion For the Christians of Tanj, the covenant of God and His Messenger, and the Muslims' sin, they are not harmed in their religion, their churches, and their lives.", 'And for the people of Sabta, among the Muslims and those who enter their religion, the same is true.']
[!] Warning: The text length exceeds the character limit of 250 for language 'en', this might cause truncated audio.
 > Processing time: 21.481252670288086
 > Real-time factor: 0.5753012424936991
 > Text splitted to sentences.
['بعد مرور هذه الدورة التدريبية ، أنت تفهم كيف يعمل AI اللطيف ، وما يمكن وما لا يمكنه فعله ، وتكون قا